In [1]:
import numpy as np
import shutil
import time
import cv2
import os
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, RandomBrightnessContrast, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, Flip, OneOf, Compose ,RandomContrast , RandomBrightness
)


In [2]:
train_folders_path = '/app/train'
val_folders_path =  '/app/validation'
train_folders_names = os.listdir(train_folders_path)
val_percent = 0.2
limit_of_augmented_files = 600
no_of_training_files = limit_of_augmented_files*(1-val_percent)

### Create validation Directory if doesn't exist


In [3]:
if not os.path.exists(val_folders_path):
    os.mkdir(val_folders_path)

for i in range(len(train_folders_names)):
    if not os.path.exists(val_folders_path +'/' + train_folders_names[i]):
        os.mkdir(val_folders_path +'/' + train_folders_names[i])


### Augment data

In [4]:
aug = Compose([
        RandomRotate90(),
        Flip(),
        Transpose(),
        OneOf([
            IAAAdditiveGaussianNoise(),
            GaussNoise(),
        ], p=0.2),
        OneOf([
            MotionBlur(p=.2),
            MedianBlur(blur_limit=3, p=.1),
            Blur(blur_limit=3, p=.1),
        ], p=0.2),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=45, p=.2),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.2),
        OneOf([
            CLAHE(clip_limit=2),
            IAASharpen(),
            IAAEmboss(),
            RandomContrast(),
            RandomBrightness(),
        ], p=0.3),
        HueSaturationValue(p=0.3),
    ], p=0.5)

for i in range(len(train_folders_names)):
    folder_name = train_folders_names[i]
    print (folder_name)
    folder_path = train_folders_path + '/' + folder_name + '/'
    files_path_list = os.listdir(folder_path)
    len_original_folder = len_augmented_folder = len(files_path_list)
    augmentation_count, image_number = 1, 0
    augment = True

    while augment:
        if len_augmented_folder >= limit_of_augmented_files:
            augment = False
            break

        image  = cv2.imread(folder_path + files_path_list[image_number])
        image_aug = aug(image=image)['image']
        filename, file_ext = os.path.splitext(os.path.basename(folder_path + files_path_list[image_number]))
        cv2.imwrite(train_folders_path + '/' + folder_name + '/' + filename + '_' + str(time.time()) + file_ext , image_aug)
        len_augmented_folder+=1
        augmentation_count+=1
        image_number +=1
        if len_augmented_folder != limit_of_augmented_files and image_number==len_original_folder:
            image_number=0 

kitchen
poolinside
greenhouse
restaurant
toystore
gym
operating_room
bedroom
airport_inside
bakery


## Rename Training Data and split data to train and validation folders:


In [5]:
for i in range(len(train_folders_names)):
    folder_name = train_folders_names[i]
    folder_path = train_folders_path + '/' + folder_name + '/'
    for count, filename in enumerate(os.listdir(folder_path)): 
        dst = folder_path+ 'img_' + str(count) + '_' + str(i) + '.jpg'
        src = folder_path + filename
        os.rename(src, dst)
        if count >= no_of_training_files:          
            shutil.move(dst,val_folders_path + '/' +folder_name+'/'+ 'img_' + str(count) + '_' + str(i) + '.jpg')
